# Extracción de Datos con YouTube API

## Instalacion de la API de Youtube

In [1]:
pip install --upgrade google-api-python-client

Note: you may need to restart the kernel to use updated packages.


## Instalación de los paquetes

In [2]:
from googleapiclient.discovery import build
from datetime import datetime
import Cuenta

## Accediendo a YouTube con la clave del API

In [3]:
youtube = build('youtube','v3',developerKey = Cuenta.getClave())

## Extrayendo datos del canal

### El canal al cuál se le van a extraer datos es este:

![Imagén del canal](Canal.jpg)

In [4]:
solicitud = youtube.channels().list(
    part='statistics',
    forHandle = '@historiasdeaves'
)
datosDelCanal = solicitud.execute()
datosDelCanal

{'kind': 'youtube#channelListResponse',
 'etag': 'KzKd1Bp_-MKtSka0hnviDgz2QiU',
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#channel',
   'etag': 'jtqxTwuZlFJwslbGenmoRNbR_Yc',
   'id': 'UCj6Y19iGJGLQdg0bPKM8l1A',
   'statistics': {'viewCount': '782077',
    'subscriberCount': '12500',
    'hiddenSubscriberCount': False,
    'videoCount': '444'}}]}

##### Nota: He usado el parametro "forHandle" en vez del "forUsername" debido a que usando este último no daba la información requerida.

## Extracción del ID del canal

In [5]:
idCanal = datosDelCanal['items'][0]['id']
idCanal

'UCj6Y19iGJGLQdg0bPKM8l1A'

## Extracción de la información de los datos de las listas de reproducción del canal

In [6]:
listasDelCanal = youtube.playlists().list(
    part = 'contentDetails,snippet', #Datos de contenido de la listas de reproduccion
    channelId = idCanal,
    maxResults=200 #Maximo resultados. Si hay más de 200 solo mostrará 200 resultados
)
resultadoListas = listasDelCanal.execute()
resultadoListas

{'kind': 'youtube#playlistListResponse',
 'etag': 'NQ-d6jliUEyZKfCqWVxpn50xei4',
 'pageInfo': {'totalResults': 14, 'resultsPerPage': 50},
 'items': [{'kind': 'youtube#playlist',
   'etag': 'ovIrPkhSsh0fIWkDL_5_7j7b0SE',
   'id': 'PLxZAFzB1EbZaaBOsL2pyDc7jCOiOiTCey',
   'snippet': {'publishedAt': '2023-05-15T19:14:23Z',
    'channelId': 'UCj6Y19iGJGLQdg0bPKM8l1A',
    'title': 'Conociendo a los Colibríes',
    'description': 'Aquí tienes todos los vídeos relacionados con los colibríes (Familia Trochilidae) y sus increíbles historias',
    'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/jU5IOmPnfLo/default.jpg',
      'width': 120,
      'height': 90},
     'medium': {'url': 'https://i.ytimg.com/vi/jU5IOmPnfLo/mqdefault.jpg',
      'width': 320,
      'height': 180},
     'high': {'url': 'https://i.ytimg.com/vi/jU5IOmPnfLo/hqdefault.jpg',
      'width': 480,
      'height': 360},
     'standard': {'url': 'https://i.ytimg.com/vi/jU5IOmPnfLo/sddefault.jpg',
      'width': 640,
  

In [7]:
totalPlaylist = []

In [8]:
while listasDelCanal is not None: #Simpre y cuando no sean nulas.
    play_list_result = listasDelCanal.execute()
    totalPlaylist += play_list_result['items']
    listasDelCanal = youtube.playlists().list_next(listasDelCanal, resultadoListas)

#Total de listas de reproduccion
print(f"Total: {len(totalPlaylist)}")

Total: 14


In [9]:
totalPlaylist

[{'kind': 'youtube#playlist',
  'etag': 'ovIrPkhSsh0fIWkDL_5_7j7b0SE',
  'id': 'PLxZAFzB1EbZaaBOsL2pyDc7jCOiOiTCey',
  'snippet': {'publishedAt': '2023-05-15T19:14:23Z',
   'channelId': 'UCj6Y19iGJGLQdg0bPKM8l1A',
   'title': 'Conociendo a los Colibríes',
   'description': 'Aquí tienes todos los vídeos relacionados con los colibríes (Familia Trochilidae) y sus increíbles historias',
   'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/jU5IOmPnfLo/default.jpg',
     'width': 120,
     'height': 90},
    'medium': {'url': 'https://i.ytimg.com/vi/jU5IOmPnfLo/mqdefault.jpg',
     'width': 320,
     'height': 180},
    'high': {'url': 'https://i.ytimg.com/vi/jU5IOmPnfLo/hqdefault.jpg',
     'width': 480,
     'height': 360},
    'standard': {'url': 'https://i.ytimg.com/vi/jU5IOmPnfLo/sddefault.jpg',
     'width': 640,
     'height': 480}},
   'channelTitle': 'Crónicas del Chivizcoyo (historias de aves)',
   'localized': {'title': 'Conociendo a los Colibríes',
    'description': 'Aqu

In [10]:
print(totalPlaylist[1]['id'])
print(totalPlaylist[1]['snippet']['title'])

PLxZAFzB1EbZY0DHDxcj8UcRte9CuqYM30
Binoculares para la observación de aves - La Mochila Pajarera


In [11]:
def timestream(time):
    time = time.replace("T","").replace("Z","") #Quitamos la T y Z del string
    date,hour = time[0:10],time[11:21] #separamos entre fecha y hora
    time = '{} {}'.format(date ,hour) #Lo concatenamos en una forma diferente
    datetime_object = datetime.strptime(time, "%Y-%m-%d %H:%M:%S") #Aplicamos una funcion de datetime para convertirlo en formato de hora
    return datetime_object #Retornamos la hora del objeto

In [12]:
#En este apartado vamos a obtener varias cosas como:
# ID, titulo de la lista de reproduccion, No. Canciones, Fecha, la imagen de portada de la lista de reproduccion
from datetime import datetime
playlist_ids = []
playlist_name = []
playlist_pictureStandard = []
playlist_timepublish = []
playlist_noVideos = []
for j in range(0,len(totalPlaylist)):
    playlist_ids.append(totalPlaylist[j]['id'])
    playlist_name.append(totalPlaylist[j]['snippet']['title'])
    playlist_pictureStandard.append(totalPlaylist[j]['snippet']['thumbnails']['standard']['url'])
    playlist_timepublish.append(timestream(totalPlaylist[j]['snippet']['publishedAt'])) #Aqui aplicamos nuestra funcion para tener un objeto datetime
    playlist_noVideos.append(totalPlaylist[j]['contentDetails']['itemCount'])

In [13]:
for i in playlist_name:
    print(i)

Conociendo a los Colibríes
Binoculares para la observación de aves - La Mochila Pajarera
Conociendo a los Chipes y Reinitas
Conociendo a las Aves Rapaces
Hablemos de Aves (directos)
Conociendo sobre la migración
Conociendo a los Mosqueros
Conociendo a los Gorriones
Conociendo a los Pájaros Carpinteros
CONSEJOS para Aprender a Observar Aves [Tips para la Observación de las Aves]
Conociendo a las Aves Playeras
Conociendo a los Búhos
Un minuto de aves
Pajareando en:


In [14]:
#Ahora aplicando el concocimiento de Pandas generemos un dataFrame con esos elementos
import pandas as pd
dataframe = {'Ids de playlists':playlist_ids,
             'Titulos de playlist':playlist_name,
             'Numero de videos':playlist_noVideos,
             'Miniatura':playlist_pictureStandard,
             'Fecha publicacion':playlist_timepublish}
df = pd.DataFrame(data=dataframe)
df

,Ids de playlists,Titulos de playlist,Numero de videos,Miniatura,Fecha publicacion
0,PLxZAFzB1EbZaaBOsL2pyDc7jCOiOiTCey,Conociendo a los Colibríes,21,https://i.ytimg.com/vi/jU5IOmPnfLo/sddefault.jpg,2023-05-15 09:14:23
1,PLxZAFzB1EbZY0DHDxcj8UcRte9CuqYM30,Binoculares para la observación de aves - La M...,13,https://i.ytimg.com/vi/RN6un44Z0HA/sddefault.jpg,2022-01-04 03:24:22
2,PLxZAFzB1EbZaVwUo6N5foohmHNVZ3bbCq,Conociendo a los Chipes y Reinitas,13,https://i.ytimg.com/vi/OikYK0QCnnk/sddefault.jpg,2021-03-10 03:48:50
3,PLxZAFzB1EbZai_izWRKCAWaOOzt3f9_h0,Conociendo a las Aves Rapaces,18,https://i.ytimg.com/vi/dZgbzB1AbiQ/sddefault.jpg,2020-12-08 01:40:33
4,PLxZAFzB1EbZbgZOxYdlbx16R79vO6Azjc,Hablemos de Aves (directos),67,https://i.ytimg.com/vi/7yIggKQF6SQ/sddefault.jpg,2020-11-07 08:34:11
5,PLxZAFzB1EbZZpaDQBL6LWhCmzXL9qam4-,Conociendo sobre la migración,6,https://i.ytimg.com/vi/BOBPnuyRpnU/sddefault.jpg,2020-10-11 00:57:30
6,PLxZAFzB1EbZa18JnhuJT5CGVYF5qkd1lZ,Conociendo a los Mosqueros,17,https://i.ytimg.com/vi/oW7CGnLH3cM/sddefault.jpg,2020-08-09 09:07:57
7,PLxZAFzB1EbZY4e9FRlDjLd0zCo0ldb267,Conociendo a los Gorriones,10,https://i.ytimg.com/vi/uEvzPjNUGMc/sddefault.jpg,2020-06-08 06:21:44
8,PLxZAFzB1EbZZ6wMbDkUKXPS-z3gT9Zbk0,Conociendo a los Pájaros Carpinteros,10,https://i.ytimg.com/vi/F0ri6AoFNkc/sddefault.jpg,2020-05-31 01:35:04
9,PLxZAFzB1EbZa5aZmsMj_lXXwsdD2rpygY,CONSEJOS para Aprender a Observar Aves [Tips p...,11,https://i.ytimg.com/vi/UejlJwt9zWM/sddefault.jpg,2020-02-18 02:59:53


In [15]:
from IPython.core.display import HTML
def a_img_tg(url):
    return '<img src="'+ url + '" width="200" >'
display(HTML(df.to_html(escape=False,formatters=dict(Miniatura=a_img_tg))))

,Ids de playlists,Titulos de playlist,Numero de videos,Miniatura,Fecha publicacion
0,PLxZAFzB1EbZaaBOsL2pyDc7jCOiOiTCey,Conociendo a los Colibríes,21,,2023-05-15 09:14:23
1,PLxZAFzB1EbZY0DHDxcj8UcRte9CuqYM30,Binoculares para la observación de aves - La Mochila Pajarera,13,,2022-01-04 03:24:22
2,PLxZAFzB1EbZaVwUo6N5foohmHNVZ3bbCq,Conociendo a los Chipes y Reinitas,13,,2021-03-10 03:48:50
3,PLxZAFzB1EbZai_izWRKCAWaOOzt3f9_h0,Conociendo a las Aves Rapaces,18,,2020-12-08 01:40:33
4,PLxZAFzB1EbZbgZOxYdlbx16R79vO6Azjc,Hablemos de Aves (directos),67,,2020-11-07 08:34:11
5,PLxZAFzB1EbZZpaDQBL6LWhCmzXL9qam4-,Conociendo sobre la migración,6,,2020-10-11 00:57:30
6,PLxZAFzB1EbZa18JnhuJT5CGVYF5qkd1lZ,Conociendo a los Mosqueros,17,,2020-08-09 09:07:57
7,PLxZAFzB1EbZY4e9FRlDjLd0zCo0ldb267,Conociendo a los Gorriones,10,,2020-06-08 06:21:44
8,PLxZAFzB1EbZZ6wMbDkUKXPS-z3gT9Zbk0,Conociendo a los Pájaros Carpinteros,10,,2020-05-31 01:35:04
9,PLxZAFzB1EbZa5aZmsMj_lXXwsdD2rpygY,CONSEJOS para Aprender a Observar Aves [Tips para la Observación de las Aves],11,,2020-02-18 02:59:53


## Datos de la duración de los videos

In [16]:
playlists_total = [] #Guardar la información de cada lista de reproducción
for i in range(0,len(playlist_ids)):
    details_playlist = youtube.playlistItems().list(
        part = 'contentDetails',
        playlistId  = playlist_ids[i], #Pedir los detalles por cada lista.
        maxResults=200 #Maximo de resultados.
    )
    playlistIdResult  = details_playlist.execute() #Hacemos la petición a Google.
    playlists_total.append(playlistIdResult) # agregamos los resultados a la lista.

In [17]:
for i in range(0,len(playlists_total)):
    for item in playlists_total[i]['items']:
        print(item)
    print()

{'kind': 'youtube#playlistItem', 'etag': 'Gr5AI7y9VoSb0GzyVqBHfZ0AMn4', 'id': 'UEx4WkFGekIxRWJaYWFCT3NMMnB5RGM3akNPaU9pVENleS5EQUE1NTFDRjcwMDg0NEMz', 'contentDetails': {'videoId': 'jU5IOmPnfLo', 'videoPublishedAt': '2021-07-07T01:00:12Z'}}
{'kind': 'youtube#playlistItem', 'etag': 'jxB2aO0FF3kpbjDIf83QZ5pUnTU', 'id': 'UEx4WkFGekIxRWJaYWFCT3NMMnB5RGM3akNPaU9pVENleS40NzZCMERDMjVEN0RFRThB', 'contentDetails': {'videoId': 'bmJpJTJ3VIk', 'videoPublishedAt': '2022-01-16T02:00:12Z'}}
{'kind': 'youtube#playlistItem', 'etag': 'x5HnibVlxwhqZQNlcACLooWA8ws', 'id': 'UEx4WkFGekIxRWJaYWFCT3NMMnB5RGM3akNPaU9pVENleS5ENDU4Q0M4RDExNzM1Mjcy', 'contentDetails': {'videoId': 'hfbMEnruecU', 'videoPublishedAt': '2018-01-15T21:53:31Z'}}
{'kind': 'youtube#playlistItem', 'etag': 'P1ZcLc-qIWT5eHQSjurylH8oHms', 'id': 'UEx4WkFGekIxRWJaYWFCT3NMMnB5RGM3akNPaU9pVENleS41MzJCQjBCNDIyRkJDN0VD', 'contentDetails': {'videoId': 'xbDM1re1EBs', 'videoPublishedAt': '2023-05-10T03:47:44Z'}}
{'kind': 'youtube#playlistItem', 'etag':

In [18]:
#Veremos que puede ser un poco grande en tener cada lista de producción y sacar
#el tiempo de reproducción manualmente por cada lista. Para solucionar este
#problema usaremos listas vacias en listas vacias, es decir, generaremos listas
#vacias entro de una lista ¿Cuántas listas vacias? Será por cada lista de reproduccion.
#El resultado son listas vacias que almacenaremos los ids de los videos por
#cada lista de reproducción
rangePlayList = [[] for _ in range(0,len(playlists_total))]
rangePlayList

[[], [], [], [], [], [], [], [], [], [], [], [], [], []]

In [19]:
#Ahora almacenaremos los elementos en cada lista vacia.
for i in range(0,len(playlists_total)):
    for item in playlists_total[i]['items']:
        vd = item['contentDetails']['videoId']
        if i==i:
            rangePlayList[i].append(vd)
rangePlayList#Lista de ids por lista de reproduccion

[['jU5IOmPnfLo',
  'bmJpJTJ3VIk',
  'hfbMEnruecU',
  'xbDM1re1EBs',
  '9IjA48HruzA',
  'OfyZLqUhfk0',
  'lzXAFw9WbiE',
  'HyyNDAvM3zg',
  'a3h5L9DCu9o',
  'l8M51-7amRw',
  'qkB-4LrJGKo',
  '7SLK1Gs6_YY',
  'KkdQfx9UG8c',
  'mo0oltR_keg',
  'JPaQotuN7fQ',
  '6S-3rh35Ewk',
  'MWXhTLj_QHU',
  '0rnNUobbgA8',
  'KOAyxWrap1Y',
  'Vo6Gbra8db4',
  'uecP1pLVXxc'],
 ['RN6un44Z0HA',
  'tmKu78ukBTY',
  'ThwL33jFFuU',
  '0y6t1bJld3A',
  'wDwuyytCHKQ',
  'FmA36NJf21A',
  'DR54BE33IUA',
  'l0izZoaguAo',
  '1QHZbfRES_A',
  'K8gas4n--3I',
  'kmQxDOtvqPE',
  'tnk83rVjkuY',
  'GMSxD8Nb3m0'],
 ['OikYK0QCnnk',
  'hgclq1yCdAA',
  'CnXrFoNHBPg',
  'DBXLZgi0tTk',
  'nuhZByxVQHo',
  '6JkTovk2gM4',
  'P6XQF9xoxyc',
  'tgicKJVHfQQ',
  '26R5T1P8oeM',
  'JkpGWmeE3BA',
  '7xzINoGBHfk',
  'iLL8EUBen4k',
  'ZILNrg1JPz8'],
 ['dZgbzB1AbiQ',
  'NLpY2-gW2XM',
  'zEmfaQrY_7A',
  'SO59R7wxGAA',
  'GE3kVPzVUlc',
  'VnYLEDqStHY',
  'L9olNxEtsso',
  'oMlp4cJtMs0',
  'CxabndOvUNc',
  'N6oGQIIq5DQ',
  'Okcxym67zzs',
  'csj8t4kA

In [20]:
#Definamos ahora una función para que sea más efectivo en que nos regrese los
#detalles de cada video por cada lista de reproducción y solamente la llamemos
#cuando sea necesario hacer una petición a Google.
def videos_reponse(id_videos):
    video_request  = youtube.videos().list(
        part='contentDetails',
        id=','.join(id_videos)
    )
    video_data = video_request.execute()
    return video_data #Regresa la información de cada video usando su id

In [21]:
playlist1_videos = videos_reponse(rangePlayList[0]) #En este ejemplo es para la lista 1 y sus videos
playlist2_videos = videos_reponse(rangePlayList[1]) #En este para la lista 2 y sus ejemplos
for item in playlist1_videos['items']:
    duration = item['contentDetails']['duration']
    print(duration)#Formato de Youtube será la duración que veremos que nos da un formato no reconocible
    print()

PT29M

PT14M52S

PT9M49S

PT3M15S

PT4M7S

PT3M39S

PT4M25S

PT3M12S

PT3M8S

PT3M28S

PT3M22S

PT3M19S

PT3M45S

PT3M

PT3M15S

PT3M8S

PT2M36S

PT2M9S

PT1M35S

PT1M23S

PT4M48S



In [22]:
#Aquí empezaremos a hablar de Expresiones Regulares
#Dejaré aquí la documentación correspondiente.
#https://docs.python.org/es/3/howto/regex.html
import re
hours_re = re.compile(r'(\d+)H')
minuntes_re = re.compile(r'(\d+)M')
seconds_re = re.compile(r'(\d+)S')
for item in playlist1_videos['items']:
    duration = item['contentDetails']['duration']
    horas = hours_re.search(duration)
    minutos = minuntes_re.search(duration)
    segundos = seconds_re.search(duration)
    print(horas,minutos,segundos)
    print()

None <re.Match object; span=(2, 5), match='29M'> None

None <re.Match object; span=(2, 5), match='14M'> <re.Match object; span=(5, 8), match='52S'>

None <re.Match object; span=(2, 4), match='9M'> <re.Match object; span=(4, 7), match='49S'>

None <re.Match object; span=(2, 4), match='3M'> <re.Match object; span=(4, 7), match='15S'>

None <re.Match object; span=(2, 4), match='4M'> <re.Match object; span=(4, 6), match='7S'>

None <re.Match object; span=(2, 4), match='3M'> <re.Match object; span=(4, 7), match='39S'>

None <re.Match object; span=(2, 4), match='4M'> <re.Match object; span=(4, 7), match='25S'>

None <re.Match object; span=(2, 4), match='3M'> <re.Match object; span=(4, 7), match='12S'>

None <re.Match object; span=(2, 4), match='3M'> <re.Match object; span=(4, 6), match='8S'>

None <re.Match object; span=(2, 4), match='3M'> <re.Match object; span=(4, 7), match='28S'>

None <re.Match object; span=(2, 4), match='3M'> <re.Match object; span=(4, 7), match='22S'>

None <re.Match o

In [23]:
#Cuando ejecutamos la celda de arriba veremos que tenemos en match = 'H'
#minutos y finalmente los segundos pero seguimos teniendo el problema en
#poder visualizarlo de manera más con detalle
#Para ello vamos a arreglar este for usando métodos que si no hay horas
#o minutos (pueden ser una lista de shorts) regrese un 0
for item in playlist1_videos['items']:
    duration = item['contentDetails']['duration']
    horas = hours_re.search(duration)
    minutos = minuntes_re.search(duration)
    segundos = seconds_re.search(duration)
    #Si no tenemos horas o minutos extensos
    horas = int(horas.group(1)) if horas else 0
    minutos = int(minutos.group(1)) if minutos else 0
    segundos = int(segundos.group(1)) if segundos else 0
    print(horas,minutos,segundos)
    print()


0 29 0

0 14 52

0 9 49

0 3 15

0 4 7

0 3 39

0 4 25

0 3 12

0 3 8

0 3 28

0 3 22

0 3 19

0 3 45

0 3 0

0 3 15

0 3 8

0 2 36

0 2 9

0 1 35

0 1 23

0 4 48



In [24]:
#Veremos que es un poco más entendible pero ahora no es tan claro.
#Vemos que tenemos que hacer esto por cada lista de reproducción,
#hagamos esto para n-videos. Definiremos una funcion de nos regrese
#el tiempo en formato de tiempo o la reproducción entre los segundos
#totales,es decir, si en una lista hay dos videos de 1hr entonces regre-
#sará 7200 segundos.
from datetime import timedelta #Transformar tiempo a segundos
def timevideosPlaylist(playlist):
    time_videos = []#Tiempo de cada video por cada lista de reproduccion.

    for item in playlist['items']:
        duration = item['contentDetails']['duration']
        horas = hours_re.search(duration)
        minutos = minuntes_re.search(duration)
        segundos = seconds_re.search(duration)
        horas = int(horas.group(1)) if horas else 0
        minutos = int(minutos.group(1)) if minutos else 0
        segundos = int(segundos.group(1)) if segundos else 0
        #Por cada video
        video_segundos = timedelta(
            hours = horas,
            minutes = minutos,
            seconds = segundos
        ).total_seconds()
        #Por cada video de la lista de reproduccion
        time_videos.append(video_segundos)
    total_reproduction = int(sum(time_videos))
    minutos,segundos =divmod(total_reproduction,60)
    horas,minutos = divmod(minutos,60)
    total_reproduction = f'{horas}:{minutos}:{segundos}' #Formato H:M:S
    return [time_videos,total_reproduction]

timevideosPlaylist(playlist2_videos) #Tenemos resultados para la segunda lista de reproduccion para cada n-video.

[[1325.0,
  752.0,
  1334.0,
  1106.0,
  1032.0,
  1021.0,
  1047.0,
  880.0,
  1451.0,
  637.0,
  651.0,
  937.0,
  1096.0],
 '3:41:9']

In [25]:
#Obtengamos la suma de cada lista resproducción con todos los videos usando la
#función que vimos arriba y aplicando esta función que nos resgresará una lista
#y cada elemento será en formato de hora de cada lista de reproducción en el canal
def timeStreamingChannels(listaTotales):
    timeStreaming_seconds = []
    for item in listaTotales:
        plt= videos_reponse(item)
        time = timevideosPlaylist(plt)[1] #Obtenemos el formato de hora.
        timeStreaming_seconds.append(time)
    return timeStreaming_seconds
timeStreamingChannels(rangePlayList) #Lo usamos para toda la lista de reproducciones del canal

['1:51:15',
 '3:41:9',
 '2:52:44',
 '2:47:49',
 '62:26:29',
 '1:13:26',
 '1:4:59',
 '0:22:53',
 '0:55:42',
 '3:18:34',
 '1:45:26',
 '1:34:19',
 '3:8:20',
 '2:47:30']

In [28]:
#Finalmente volveremos a correr el código  y añadiendo una columna donde podamos
#visualizar el tiempod de duración de cada lisa de reproducción en total
duration = timeStreamingChannels(rangePlayList)
df['Duracion'] = duration #Agregamos una nueva columna con los datos al dataFrame
from IPython.core.display import HTML
def a_img_tg(url):
    return '<img src="'+ url + '" width="175" >'
display(HTML(df.to_html(escape=False,formatters=dict(Miniatura=a_img_tg))))

,Ids de playlists,Titulos de playlist,Numero de videos,Miniatura,Fecha publicacion,Duracion
0,PLxZAFzB1EbZaaBOsL2pyDc7jCOiOiTCey,Conociendo a los Colibríes,21,,2023-05-15 09:14:23,1:51:15
1,PLxZAFzB1EbZY0DHDxcj8UcRte9CuqYM30,Binoculares para la observación de aves - La Mochila Pajarera,13,,2022-01-04 03:24:22,3:41:9
2,PLxZAFzB1EbZaVwUo6N5foohmHNVZ3bbCq,Conociendo a los Chipes y Reinitas,13,,2021-03-10 03:48:50,2:52:44
3,PLxZAFzB1EbZai_izWRKCAWaOOzt3f9_h0,Conociendo a las Aves Rapaces,18,,2020-12-08 01:40:33,2:47:49
4,PLxZAFzB1EbZbgZOxYdlbx16R79vO6Azjc,Hablemos de Aves (directos),67,,2020-11-07 08:34:11,62:26:29
5,PLxZAFzB1EbZZpaDQBL6LWhCmzXL9qam4-,Conociendo sobre la migración,6,,2020-10-11 00:57:30,1:13:26
6,PLxZAFzB1EbZa18JnhuJT5CGVYF5qkd1lZ,Conociendo a los Mosqueros,17,,2020-08-09 09:07:57,1:4:59
7,PLxZAFzB1EbZY4e9FRlDjLd0zCo0ldb267,Conociendo a los Gorriones,10,,2020-06-08 06:21:44,0:22:53
8,PLxZAFzB1EbZZ6wMbDkUKXPS-z3gT9Zbk0,Conociendo a los Pájaros Carpinteros,10,,2020-05-31 01:35:04,0:55:42
9,PLxZAFzB1EbZa5aZmsMj_lXXwsdD2rpygY,CONSEJOS para Aprender a Observar Aves [Tips para la Observación de las Aves],11,,2020-02-18 02:59:53,3:18:34


## Encontrando los videos con más likes

In [34]:
#Vamos a definir las dos funciones faltantes para obtener este resultado.
#La primera nos hará la petición por cada video de sus estadísticas y propiedades de
#estilo o dieseño. La segundo será para obtener los likes en los videos.

#Estadisticas por cada video
def stadistics_video(id_videos):
    video_request  = youtube.videos().list(
        part='statistics,snippet',
        id=','.join(id_videos)
    )
    video_data = video_request.execute()
    return video_data
#Solucion por decorador
def likesvideos_per_playlist(function):
    video_view = []#Tiempo de cada video por cada lista de reproduccion.
    for item in function['items']:
        likes = item['statistics']['likeCount'] #Likes de videos
        video_id = item['id'] #Id del video
        name_video = item['snippet']['title'] #Titulo el video
        yt_link = f'https://youtu.be/{video_id}'#Link del video
        video_view.append({'title':name_video,
                            'likeCount':int(likes),
                           'url':yt_link}) #Diccionario donde se almacen los datos anteriores
    video_view.sort(key=lambda vid:vid['likeCount'],reverse=True) #Este método lo que hacemos es ordenarlos de mayor a menor todos sus elementos
    return video_view #Retornamos el diccionario con los videos ordenados con mayores likes por cada lista de reproducción

In [35]:
pl1_staditic  = stadistics_video(rangePlayList[0])
view_plst = likesvideos_per_playlist(pl1_staditic)

In [39]:
#Si nos damos cuenta de algo, es que como regrese un diccionario podemos jugar
#con los indices obteniendo los likes, el link y titulo como podemos ver aquí
view_plst[12]['likeCount'],view_plst[12]['url'],view_plst[12]['title']

(87,
 'https://youtu.be/uecP1pLVXxc',
 'El misterioso Ermitaño de la selva | #UnMinutodeAves No.183')

In [40]:
#Vamos organizarlo de una manera mejor y finalmente que quede indexado en nuetro dataframe
def mostPopularSongbyPL(listasCanal):
    name_popularvid = [] #Lista de los videos más populares
    liked_list = [] #Lista de likes
    url_list = [] #Urls de los videos
    noneHit = None #En el caso que no hubiera información de ello
    for i in listasCanal:
        staditica = stadistics_video(i) #Llamamos la función de arriba
        try: #Si hay elementos disponibles llenamos las listas
            liked = likesvideos_per_playlist(staditica)
            name_popularvid.append(liked[0]['title'])
            liked_list.append(liked[0]['likeCount'])
            url_list.append(liked[0]['url'])
        except:# En el caso que no hubiera un elemento(s) para tratar ese error se hará listas nulas como se observará
            name_popularvid.append(noneHit)
            liked_list.append(noneHit)
            url_list.append(noneHit)
    return [name_popularvid,liked_list,url_list] #Regresamos los nombres,likes y las urls de los videos
mostPopularSongbyPL(rangePlayList)

[['Colibríes | Entre la guerra y las flores',
  'Aprendiendo a Observar Aves 1.4 | Usando Binoculares + Review de modelos',
  'La alucinante historia del Chipe Rojo',
  'Águila Arpía | La Reina de la Selva',
  'Hablemos de aves 59 no se de que hablar hoy, vengo llegando de pajarear',
  'Aves Playeras | Las Reinas de la Migración',
  'La tierra de los Tiranos | La incomprensible historia de los mosqueros',
  'Un Minuto de Aves #73 Gorrión Serrano',
  'Carpintero Imperial | El último de los gigantes',
  'Aprendiendo a observar aves 1.2 | ¿Cómo las identifico? parte 1',
  'Aves Playeras | Las Reinas de la Migración',
  'Búhos y Lechuzas|¿Acaso son diferentes?',
  '¿es esta el ÁGUILA de la bandera 🇲🇽? | #UnMinutodeAves No. 189',
  'Pajareando en: Península de Yucatán'],
 [504, 696, 258, 623, 92, 250, 200, 130, 1136, 805, 250, 577, 129, 608],
 ['https://youtu.be/jU5IOmPnfLo',
  'https://youtu.be/RN6un44Z0HA',
  'https://youtu.be/26R5T1P8oeM',
  'https://youtu.be/dZgbzB1AbiQ',
  'https://you

In [30]:
#Para visualizar los links se puedan dar click a que hacer una pequeña modificacion
#Pero primero indexamos los elementos al dataFrame que teníamos
df['Nombre de Hit'],df['Likes'],df['UrlHit'] = mostPopularSongbyPL(rangePlayList)[0],mostPopularSongbyPL(rangePlayList)[1],mostPopularSongbyPL(rangePlayList)[2]
display(HTML(df.to_html(escape=False,render_links=True,formatters=dict(Miniatura=a_img_tg)))) #render_links = True lo permite.

,Ids de playlists,Titulos de playlist,Numero de videos,Miniatura,Fecha publicacion,Duracion,Nombre de Hit,Likes,UrlHit
0,PLxZAFzB1EbZaaBOsL2pyDc7jCOiOiTCey,Conociendo a los Colibríes,21,,2023-05-15 09:14:23,1:51:15,Colibríes | Entre la guerra y las flores,504,https://youtu.be/jU5IOmPnfLo
1,PLxZAFzB1EbZY0DHDxcj8UcRte9CuqYM30,Binoculares para la observación de aves - La Mochila Pajarera,13,,2022-01-04 03:24:22,3:41:9,Aprendiendo a Observar Aves 1.4 | Usando Binoculares + Review de modelos,696,https://youtu.be/RN6un44Z0HA
2,PLxZAFzB1EbZaVwUo6N5foohmHNVZ3bbCq,Conociendo a los Chipes y Reinitas,13,,2021-03-10 03:48:50,2:52:44,La alucinante historia del Chipe Rojo,258,https://youtu.be/26R5T1P8oeM
3,PLxZAFzB1EbZai_izWRKCAWaOOzt3f9_h0,Conociendo a las Aves Rapaces,18,,2020-12-08 01:40:33,2:47:49,Águila Arpía | La Reina de la Selva,623,https://youtu.be/dZgbzB1AbiQ
4,PLxZAFzB1EbZbgZOxYdlbx16R79vO6Azjc,Hablemos de Aves (directos),67,,2020-11-07 08:34:11,62:26:29,"Hablemos de aves 59 no se de que hablar hoy, vengo llegando de pajarear",92,https://youtu.be/yAfnMgi6tiw
5,PLxZAFzB1EbZZpaDQBL6LWhCmzXL9qam4-,Conociendo sobre la migración,6,,2020-10-11 00:57:30,1:13:26,Aves Playeras | Las Reinas de la Migración,250,https://youtu.be/QhX4hYCOoJw
6,PLxZAFzB1EbZa18JnhuJT5CGVYF5qkd1lZ,Conociendo a los Mosqueros,17,,2020-08-09 09:07:57,1:4:59,La tierra de los Tiranos | La incomprensible historia de los mosqueros,200,https://youtu.be/oW7CGnLH3cM
7,PLxZAFzB1EbZY4e9FRlDjLd0zCo0ldb267,Conociendo a los Gorriones,10,,2020-06-08 06:21:44,0:22:53,Un Minuto de Aves #73 Gorrión Serrano,130,https://youtu.be/4gbPsFYUe50
8,PLxZAFzB1EbZZ6wMbDkUKXPS-z3gT9Zbk0,Conociendo a los Pájaros Carpinteros,10,,2020-05-31 01:35:04,0:55:42,Carpintero Imperial | El último de los gigantes,1136,https://youtu.be/VLY8rVIAtE8
9,PLxZAFzB1EbZa5aZmsMj_lXXwsdD2rpygY,CONSEJOS para Aprender a Observar Aves [Tips para la Observación de las Aves],11,,2020-02-18 02:59:53,3:18:34,Aprendiendo a observar aves 1.2 | ¿Cómo las identifico? parte 1,805,https://youtu.be/clU2EbKAuTg
